In [1]:
import selenium
import bs4 

print(bs4.__version__)
print(selenium.__version__)

4.11.1
4.4.3


In [1]:
import os 
from pathlib import Path
from selenium import webdriver

path_to_driver = Path(os.getenv('PYPJ')) / 'Crowler/driver/chromedriver_win32/chromedriver.exe'

from linkedin.util.crowler import Crowler


### Crowler Class Implementation 

In [ ]:
import os
import re 
import time
from xml.dom.minidom import Element 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.remote.webelement import WebElement
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions

path_to_linkedin = 'https://linkedin.com'
username = 'frusciante2580@gmail.com'
password = 'Utatistics0511'
search_word = 'data scientist'
path_to_output = os.getcwd() + '\\..\\..\\output\\'

class Crowler:
    def __init__(self, path_to_driver: str, target_url: str, path_to_output: str) -> None:
        self.path_to_driver = Service(path_to_driver)
        self.target_url = target_url
        self.path_to_output = path_to_output

        self.driver = webdriver.Chrome(service=self.path_to_driver)
        self.wait = WebDriverWait(self.driver, 30)

    def login(self, username: str, password: str) -> None:
        self.username = username
        self.password = password

        self.driver.get(self.target_url)
        self.driver.maximize_window()

        self.driver.find_element(By.CLASS_NAME,'nav__button-secondary').click()
        self.driver.find_element(By.ID, 'username').send_keys(self.username)
        self.driver.find_element(By.ID, 'password').send_keys(self.password)
        self.driver.find_element(By.CLASS_NAME, 'login__form_action_container ').click()
    
    def search_jobs(self, search_word: str) -> None:
        self.search_word = search_word
        print(f'searching the jobs: {self.search_word}')
  
        self.wait.until(expected_conditions.element_to_be_clickable((By.CLASS_NAME,'search-global-typeahead__input')))
        self.driver.find_element(By.CLASS_NAME, 'search-global-typeahead__input').send_keys(self.search_word)
        self.driver.find_element(By.CLASS_NAME, 'search-global-typeahead__input').send_keys(Keys.ENTER)

        while True:
            cnt = 1
            self.wait.until(expected_conditions.element_to_be_clickable((By.XPATH,f'//*[@id="main"]/div/div/div[{cnt}]/div[2]/a')))
            element = self.driver.find_element(By.XPATH, f'//*[@id="main"]/div/div/div[{cnt}]/div[2]/a')
            if 'job' in element.text:
                element.click()
                break 
            else:
                cnt += 1
                if cnt > 5:
                    print('Jobs not found.')
                    break
    
    def get_urls(self) -> None:
        self._implicitly_wait(3)
        
        page_cnt = 1
        while True:
            self.urls = []
            self.driver.find_element(By.CLASS_NAME, 'scaffold-layout__list-container')
            elements = self.driver.find_elements(By.CLASS_NAME, 'jobs-search-results__list-item')
            print(f'{len(elements)} elements found in page {page_cnt}.')

            for element in elements:
                try:
                    self._url_getter(element)
                
                except NoSuchElementException:
                    self.driver.execute_script('arguments[0].scrollIntoView({behavior: "smooth", block: "center"});', element)
                    self._implicitly_wait(1)
                    self._url_getter(element)
            
            page_cnt += 1
            pages = self.driver.find_element(By.CLASS_NAME, 'artdeco-pagination__pages').find_elements(By.TAG_NAME, 'li')
            eop_status = self._page_turnner(page_cnt=page_cnt, elements=pages)
            if eop_status:
                break 
        
        print(f'collected {len(self.urls)} jobs.')
        
        return self.urls

    def browse_job(self) -> None:
        self.job_html_dict = {}
        dirname = self.path_to_output + self.search_word.replace(' ', '_')
        os.system(f"mkdir {dirname}")

        for url in self.urls:
            self.driver.get(url)
            self._job_info_getter()
            title = self._job_text_getter()
            # self._job_innerHTML_getter()
            self._save_to_txt(title)

    def _page_turnner(self, page_cnt: int, elements: WebElement) -> bool:
        iter_cnt = 0
        for page in elements:
            page_num = page.find_element(By.TAG_NAME, 'button').text        
            if page_num.isdigit(): page_num = int(page_num)
    
            if page_num == page_cnt:
                page.click()
                eop_status = False
                break

            elif page_num == '…' and iter_cnt > 1:
                page.click()
                eop_status = False
                break
               
            else:
                eop_status = True
            
            iter_cnt += 1
        
        print('page number: %s' % str(page_num))
        return eop_status  
    
    def _url_getter(self, element: WebElement) -> None:
        url = element.find_element(By.TAG_NAME, 'a').get_attribute('href')
        self.urls.append(url)
        
    def _job_info_getter(self) -> None:
        self._implicitly_wait(1)
        classname = {
            'title': 'jobs-unified-top-card__job-title',
            'company_name': 'jobs-unified-top-card__company-name',
            'location': 'jobs-unified-top-card__bullet',
            'posted_date': 'jobs-unified-top-card__posted-date',
            'applicants' : 'jobs-unified-top-card__applicant-count'}
              
        self.data_job_info = {}
        for key in classname.keys():
            try:
                identifier = classname[key]
                src_get = f"element_{key} = self.driver.find_element(By.CLASS_NAME, '{identifier}')" 
                src_append = f"self.data_job_info['{key}'] = element_{key}.text" 
                exec(src_get)
                exec(src_append)

            except NoSuchElementException:
                print(f'{key} not found.')
                src_append = f"'self.data_job_info['{key}'] = None"
                exec(src_append)

    def _job_icon_getter(self) -> None:
        self._implicitly_wait(1)
        
    def _job_innerHTML_getter(self) -> str:
        self._implicitly_wait(1)
        html_data = self.driver.find_element(By.CLASS_NAME,'jobs-box__html-content').get_attribute('innerHTML')
        title = self.data_job_info['title'] + '_' +  self.data_job_info['company_name']
        self.job_html_dict[title] = html_data
        return title

    def _job_text_getter(self) -> str:
        self._implicitly_wait(1)
        text_data = self.driver.find_element(By.CLASS_NAME,'jobs-box__html-content').text
        title = self.data_job_info['title'] + '_' +  self.data_job_info['company_name']
        self.job_html_dict[title] = text_data
        return title
    
    def _implicitly_wait(self, secs: int) -> None:
        time.sleep(secs)

    def _save_to_txt(self, title) -> None:
        dirname = self.path_to_output + self.search_word.replace(' ', '_')
        titlename = re.sub(r'[\\|/|:|?|.|"|<|>|\|]', '-', title) 
        filename =  dirname + '\\' + titlename.replace(' ','') + '.txt'
        with open(file=filename, mode='w', encoding='utf-8') as f:
            f.writelines(self.job_html_dict[title])

    def run(self, username: str, password: str, search_word: str) -> None:
        print('login...')
        self.login(username=username, password=password)
        self.search_jobs(search_word=search_word)
        
        print('collecting the URLs...')
        result = self.get_urls()
        
        print(f'browsing the job: place_holder')
        self.browse_job()
        
    def exit(self) -> None:
        time.sleep(3)
        self.driver.quit()


clowler = Crowler(path_to_driver=path_to_driver, target_url=path_to_linkedin, path_to_output=path_to_output)
results = clowler.run(username=username, password=password, search_word=search_word)



### DbMaker implementation

In [99]:
import os 
import csv
import json
import re
from re import Pattern

path_to_output = os.getcwd() + '\\..\\..\\output\\'
search_word = 'data scientist'

class DbMaker:
    def __init__(self, path_to_output: str, search_word: str) -> None:
        self.path_to_output = path_to_output + search_word.replace(' ', '_') + '\\'
        self.path_to_txt = path_to_output + search_word.replace(' ', '_') + '\\txt\\'
        self.path_to_json = path_to_output + search_word.replace(' ', '_') + '\\json\\'
        self.dbname = path_to_output + search_word.replace(' ', '_') + '\\database.csv'
        self.num_of_entries = len(os.listdir(self.path_to_txt))
        self.encoding = 'utf-8'
        self.newline = ""
    
    def create_csv(self, colnames: list) -> None:
        self.default_keys = ['filename', 'title','company_name', 'location','posted_date', 'applicants', 'salary']
        self.colnames = colnames
        self.header = [*self.default_keys, *self.colnames]
        self.header.append('Salary')

        with open(file=self.dbname, mode='w', encoding=self.encoding, newline=self.newline) as f:
            writer = csv.writer(f)
            writer.writerow(self.header)
    
    def write_to_csv(self) -> None:
        with open(file=self.dbname, mode='a', encoding=self.encoding, newline=self.newline) as f:
            writer = csv.writer(f)

            for jsonfile in os.listdir(self.path_to_json):
                txtfile = jsonfile.replace('json','txt')
                row = self._write_job_info(jsonfile)
                row = self._write_job_contents(txtfile, row)
                writer.writerow(row)

    def _write_job_info(self, jsonfile: str) -> list:
        row = [jsonfile.replace('.json','')]
        with open(file=self.path_to_json + jsonfile, mode='r',encoding=self.encoding) as fp:
            job_info_dict = json.load(fp)
            for key in job_info_dict.keys():
                row.append(job_info_dict[key])
            return row
            
    def _write_job_contents(self, txtfile: str, row: list) -> list:
        with open(file=self.path_to_txt + txtfile, mode='r', encoding=self.encoding) as f:
            job_info_txt = f.readlines()
            compiled = re.compile('\D*\d\d,000\D*')

            for word in self.colnames:
                status = False
                row = self._get_bool_info(row, job_info_txt, status, word)
            
            row = self._get_saraly(row, job_info_txt, compiled)

        return row
    
    def _get_bool_info(self, row: list, job_info_txt: list, status: bool, word: str) -> list:
        for line in job_info_txt:
            if word in line:
                status = True
                break
        row.append(status)
        return row 

    def _get_saraly(self, row: list, job_info_txt: list, compiled: Pattern) -> list:
        for line in job_info_txt:
            matched  = compiled.match(line)
            try:
                salary = re.sub(r'\D', '', matched.group())
                row.append(salary)
                break
            except AttributeError:
                pass
        
        row.append(None)
        return row

    def run(self, colnames: list):
        self.create_csv(colnames)
        self.write_to_csv()
 

colnames = ['Python', 'SQL', 'SAS', 'C++','MSc', 'PhD', 'Cloud', 'AWS', 'Azure', 'Statistics','Computer Science', 'CS', 'consulting'] 
dbmaker = DbMaker(path_to_output=path_to_output, search_word=search_word)
dbmaker.run(colnames)
